# > O problema do Caixeiro Viajante (Traveling Salesman Problem - TSP) utilizando Algoritmos Genéticos

Atividade de GCC128 - Inteligência Artificial

Aluno: Thiago Odilon de Almeida - 202021025

Seguindo as diretrizes de Implementação, primeiro criamos uma função para gerar uma população inicial de rotas de forma aleatória.

In [ ]:
import numpy as np

In [ ]:
def geraPopulacao(nCidades, tamPopulacao):
    populacao = []
    for i in range(tamPopulacao):
        rota = np.random.permutation(nCidades)
        populacao.append(rota)
    return populacao

Logo em seguida foi feita a implementação da seleção por torneio que também faz uso de outras duas funções que são: ***calcularDist***, que faz o cálculo de uma cidade a outra usando a fórmula de distância Euclidiana.

In [ ]:
def calcularDist(c1, c2):
  a = (c1[0] - c2[0]) ** 2
  b = (c1[1] - c2[1]) ** 2
  return np.sqrt(a + b)

 A função calcularDistTotal que faz o cálculo total da rota, na seleção da ***cidadeProx*** é verificada a condição se ***i*** é o último elemento, caso seja ele volta para o primeiro elemento do vetor, caso não ele percorre normalmente.


In [ ]:
def calcularDistTotal(rota, cidades):
  distTotal = 0
  for i in range(len(rota)):
    cidadeSelecionada = cidades[rota[i]]
    if i == len(cidades) - 1:
      cidadeProx = cidades[rota[0]]
    else:
      cidadeProx = cidades[rota[i + 1]]
    distTotal += calcularDist(cidadeSelecionada, cidadeProx)

  return distTotal

Como estratégia de implementação da seleção do torneio, primeiro é atribuído à variável ***competidores*** elementos aleatórios da ***populacao*** restrigido ao tamanho de ***tamTorneio***. Em seguida é calculado o valor de aptidão para cada competidor selecionado e por fim é atribuído ao ***vencedor*** o menor valor na lista com o auxílio da função ***argmin*** e adiciona o atual vencedor ao vetor de ***pais***.

In [ ]:
import random

In [ ]:
def selecaoTorneio(populacao, cidades, tamTorneio):
    pais = []
    for i in range(len(populacao)):
        competidores = random.sample(populacao, tamTorneio)
        competidoresFitness = []
        for rota in competidores:
          fitness = calcularDistTotal(rota, cidades)
          competidoresFitness.append(fitness)
        vencedor = competidores[np.argmin(competidoresFitness)]
        pais.append(vencedor)
    return pais

Feita a seleção, agora partimos para a função de ***crossover*** para criar rotas descendentes combinando segmentos de duas rotas pais. Dessa forma é criada a repetição para que percorra os pais de dois em dois, atribui o valor as variáveis p1 e p2, logo em seguida é criado os filhos do tamanho dos pais com os mesmos valores fazendo o uso da função ***copy*** e após isso é feito o corte e é chamada a função ***preencheNums*** para que seja alterado os valores dos filhos desde que não se repita as cidades.

In [ ]:
def preencheNums(f, p, pc):
  nCopy = f[pc:]
  nRestantes = np.setdiff1d(p, nCopy)
  for i, num in enumerate(f[:pc]):
      if num in nCopy:
          posNum = np.where(nCopy == num)[0]
          if len(posNum) > 0:
              posNum  = posNum [0]
              nEscolhido = nRestantes[posNum]
              f[i] = nEscolhido

def crossover(pais):
  filhos = []
  for i in range(0, len(pais), 2):
    p1 = pais[i]
    p2 = pais[i + 1]
    f1 = np.copy(p1)
    f2 = np.copy(p2)

    pCorte = np.random.randint(1, len(p1) - 1)
    f1[pCorte:] = p1[pCorte:]
    f2[pCorte:] = p2[pCorte:]

    preencheNums(f1, p1, pCorte)
    preencheNums(f2, p2, pCorte)
    filhos.extend([f1, f2])

  return filhos

Agora iremos implementar o operador de mutação para introduzir pequenas alterações aleatórias na ordem das cidades dentro de uma rota.

In [ ]:
def mutacao(filhos, taxa):
  for i in range(len(filhos)):
    if(random.random() < taxa):
      filho = filhos[i]
      pos = random.sample(range(1, len(filho)), 2)
      pos1, pos2 = pos[0], pos[1]
      temp2 = filho[pos1]
      filho[pos1] = filho[pos2]
      filho[pos2] = temp2

  return filhos

E agora por fim, devemos criar a função do ***tspGenetico*** que de início faz a geração da população e logo em seguida cria as variáveis de ***melhorRota*** e ***melhorDistancia***, logo após é feito um loop que percorre o ***numGeracoes***, a função ***selecaoTorneio*** é chamada para selecionar os pais para a próxima geração. Em seguida a função ***crossover*** é chamada para fazer o cruzamento dos pais selecionados, subsequentemente a função ***mutacao*** é chamada para aplicar a mutação aos filhos gerados pelo cruzamento. E agora é percorrida cada rota na população atual, em que se a distância for menor do que a melhor (que é um número altíssmo em primeira instância) eles são atualizados.

In [ ]:
def tspGenetico(cidades, tamPopulacao, numGeracoes, tamTorneio, taxa):
  populacao = geraPopulacao(len(cidades), tamPopulacao)
  melhorRota = None
  melhorDistancia = 99999999

  for geracao in range(numGeracoes):
    pais = selecaoTorneio(populacao, cidades, tamTorneio)
    filhos = crossover(pais)
    filhosMutados = mutacao(filhos, taxa)

    for rota in filhosMutados:
      distancia = calcularDistTotal(rota, cidades)
      if(distancia < melhorDistancia):
        melhorRota = rota
        melhorDistancia = distancia
        print("Rota e distancia selecionada como a melhor no momento atual: ", melhorRota, "->", melhorDistancia)

  return melhorRota, melhorDistancia

Feito o algoritmo vamos aos testes e análise dos resultados, como mostrado no caso a baixo o algoritmo por meio da funcionalidade do algoritmo genético foi alterando as cidades no ***crossover*** utilizando de um ponto de corte e na mutação de um fator aleatório que é comparado com a nossa ***taxa***.

In [ ]:
cidades = np.random.rand(20,2)
rota, distancia = tspGenetico(cidades, 100, 200, 5, 0.05)
print("Melhor combinação de rota/distância obtida: ", rota, "->", distancia)

Rota e distancia selecionada como a melhor no momento atual:  [ 3 13  6 18  4 12  0 16 17 19  2  9 14 15  7  1 11  5 10  8] -> 9.267486796538094
Rota e distancia selecionada como a melhor no momento atual:  [18 14  5 19  8 10  2  3  7 11 15  0 13  6  9 17 12  4 16  1] -> 9.086898030223908
Rota e distancia selecionada como a melhor no momento atual:  [13  3  8 14 12 15  6  9  0  1  2 10  7 17 18  4 11 16  5 19] -> 8.889565562254413
Rota e distancia selecionada como a melhor no momento atual:  [10  7 19  3 18 12  9  2  0 14 13  6 17  4 11 16  8 15  5  1] -> 8.52274545538634
Rota e distancia selecionada como a melhor no momento atual:  [13  3  8 14 12  9  6 15  0  1  2 10  7 17 18  4 11 16  5 19] -> 8.220431888525813
Rota e distancia selecionada como a melhor no momento atual:  [10  7 19  3 18 12  9 17  0 14 13  6 15  4 11 16  8  2  5  1] -> 7.861285456022091
Rota e distancia selecionada como a melhor no momento atual:  [18  4  7 15 17  3  5 16 11  2  1  0 10  8 19  9  6 13 12 14] -> 7.43

Algo interessante que podemos testar nesse segundo caso é que aumentar o tamanho do torneio (5 -> 10) favorece indivíduos mais fortes, enquanto reduzir o tamanho do torneio proporciona uma melhor chance para que indivíduos mais fracos sejam selecionados como pais.

In [ ]:
cidades = np.random.rand(20,2)
rota, distancia = tspGenetico(cidades, 100, 200, 10, 0.05)
print("Melhor combinação de rota/distância obtida: ", rota, "->", distancia)

Rota e distancia selecionada como a melhor no momento atual:  [ 0 19 15  9  7  3 10  6 12 14  2  1 18  4 17 13 16  8  5 11] -> 7.688296188074146
Rota e distancia selecionada como a melhor no momento atual:  [11  2  0  5 10  7  9 18 14  6 16  8 12  1 13 17  3 19  4 15] -> 7.220784640510772
Rota e distancia selecionada como a melhor no momento atual:  [11  2  0  5 10  7  3 18 14  6 16  8 12  1 13 17  9 19  4 15] -> 7.165699644610236
Rota e distancia selecionada como a melhor no momento atual:  [ 2  0  5 18 10 12  3  9  4 19  8 16 14  6 13  1 17 15  7 11] -> 6.540464778279621
Rota e distancia selecionada como a melhor no momento atual:  [11  2  0  5 10  7  1 18 14  6 16  8 12  3 13 17  9 19  4 15] -> 6.4633494845746355
Melhor combinação de rota/distância obtida:  [11  2  0  5 10  7  1 18 14  6 16  8 12  3 13 17  9 19  4 15] -> 6.4633494845746355


De forma similar, se aumentarmos o tamanho da população (100 -> 150) isso pode permitir uma busca mais ampla no espaço de soluções e de mesma forma aumentar o número de gerações (200 -> 280) permite que o algoritmo evolua por mais interações, potencialmente melhorando a qualidade das soluções encontradas, mas com isso é também aumentado o tempo de execução do algoritmo.

In [ ]:
cidades = np.random.rand(20,2)
rota, distancia = tspGenetico(cidades, 150, 280, 5, 0.05)
print("Melhor combinação de rota/distância obtida: ", rota, "->", distancia)

Rota e distancia selecionada como a melhor no momento atual:  [11  1  3  6  4 13  0  7 18  2  5 19 17 14  9 15 10 16  8 12] -> 9.29706386748111
Rota e distancia selecionada como a melhor no momento atual:  [12  4 15 10  0  6 11  2 14 17  7 18  3 16 13  5  9  8 19  1] -> 8.682721892100693
Rota e distancia selecionada como a melhor no momento atual:  [ 9  0 16  5 14 12  3 19  1  8  6 18  4 15 11 13  2  7 10 17] -> 8.17719339690417
Rota e distancia selecionada como a melhor no momento atual:  [ 9  0  2  5 14 12  3 19  1  8  6 18  4 15 11 13 16  7 10 17] -> 7.537533691941363
Rota e distancia selecionada como a melhor no momento atual:  [19  9 18  2  1  0 15  4 17 10 16 13  7  6 12  8  5 14 11  3] -> 7.251703012176033
Rota e distancia selecionada como a melhor no momento atual:  [ 6  5 16 10  7 13 18  1  9 14 12  8 19  3  0 17  2 11 15  4] -> 7.198355677989397
Melhor combinação de rota/distância obtida:  [ 6  5 16 10  7 13 18  1  9 14 12  8 19  3  0 17  2 11 15  4] -> 7.198355677989397


E por fim, podemos também alterar o parâmetro de taxa de mutação (0.05 -> 0.2) que controla a probabilidade de ocorrer mutações nas rotas durante o processo evolutivo, com uma taxa de mutação mais alta, permitimos ao algoritmo uma exploração mais ampla do espaço de soluções, enquanto uma mais baixa pode permitir uma maior exploração das soluções atuais.

In [ ]:
cidades = np.random.rand(20,2)
rota, distancia = tspGenetico(cidades, 100, 200, 5, 0.2)
print("Melhor combinação de rota/distância obtida: ", rota, "->", distancia)

Rota e distancia selecionada como a melhor no momento atual:  [ 9  6 13 12 15 14 11  3 16  8  5  2  4  0 18 19  7 10  1 17] -> 8.722995274289772
Rota e distancia selecionada como a melhor no momento atual:  [ 9  7 13  4  1 14  3 16 18 15  8 12  5  6 19 10 17 11  2  0] -> 7.728782085415925
Rota e distancia selecionada como a melhor no momento atual:  [ 4  1 16 12 17 13  5  2  9 18  0  7 11  6  3  8 10 15 19 14] -> 7.581877099800215
Rota e distancia selecionada como a melhor no momento atual:  [ 9 13  7  4  1 14  3 16 18 15  8 12  5  6 19 10 17 11  2  0] -> 7.499079205724361
Rota e distancia selecionada como a melhor no momento atual:  [14  4  0 13  9 11  7 17  3 19  8 18  6  1 15 10 12  5 16  2] -> 6.63602443179168
Rota e distancia selecionada como a melhor no momento atual:  [14  4  0 13  6 11  7 17  3 19  8 18 12  1 15 10  9  5 16  2] -> 6.608283234368816
Melhor combinação de rota/distância obtida:  [14  4  0 13  6 11  7 17  3 19  8 18 12  1 15 10  9  5 16  2] -> 6.608283234368816


# > Conclusão

O algoritmo genético tem sido eficaz na obtenção de alta qualidade para resolver o problema do TSP. No entando, a eficácia do algortimo pode variar dependendo das características específicas do problema, como o número de cidades, a complexidade das rotas e a presença de restrições adicionais.

Uma das principais vantagens do algoritmo genético é a capacidade de explorar um amplo espaço de soluções e encontrar soluções promissoras, isso ocorre devido à combinação de operadores genéticos, como seleção por torneio, crossover e mutação, que permitem a criação de novas soluções a partir de soluções existentes. Essa capacidade de explorar diferentes soluções ajuda a evitar ficar preso em mínimos locais e a encontrar soluções melhores.

No entanto, um desafio enfrentado pelo algoritmo genético é a possibilidade de convergência prematura, na qual a população converge para uma solução sub-ótima antes de explorar todo espaço de busca. Para evitar a convergência prematura é possível ajustar os parâmetros do algoritmo, eles podem ajudar a manter a diversidade genética da população, permitindo a exploração contínua de diferentes soluções.

Com isso podemos concluir que os parâmetros devem ser ajustados dependendo do tamanho do problema, da complexidade das rotas e do tempo disponível, sendo que para alguns casos genéricos temos as seguintes quantidades para cada parâmetro:

- Tamanho da população (***tamPopulacao***): geralmente entre 50 e 200 dependendo do tamanho.
- Numero de gerações (***numGeracoes***): varia de acordo com a complexidade do problema e tempo disponível, valores entre 100 e 1000 são comuns.
- Tamanho do torneio (***tamTorneio***): valores entre 3 e 10 costumam ser adequados, valores menor favorecem diversidade genética, enquanto valores maiores favorecem a exploração de soluções mais promissoras.
- Taxa de mutação (***taxa***): geralmente fica entre 0.01 e 0.2, valores mais altos aumentam a diversidade da população enquanto valores mais baixos favorecem a exploração de soluções existentes.

# > Referências

- https://revistas.unibh.br/dcet/article/view/304/163

- https://revistas.anchieta.br/index.php/RevistaUbiquidade/article/view/2031/1754

- https://sites.icmc.usp.br/andre/research/genetic/

- http://www.nce.ufrj.br/GINAPE/VIDA/alggenet.htm

- https://www.youtube.com/watch?v=uQj5UNhCPuo
